In [1]:
import json
import numpy as np
import cv2
import os
import re
from PIL import Image
from IPython.display import display

import abrupt_transition_detection as atd
import common

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [2]:
with open(common.ABRUPT_TRANSITION_TEST) as f:
    at_data = json.load(f)

arr = np.load(common.SCENE_TRANSITION_NPZ)
pix_abs = arr['pix_abs']
hist_all = arr['hist_all']
hist_reg = arr['hist_reg']

In [3]:
model_dict = {
    'abs': {
        'kmeans': KMeans(n_clusters=2, random_state=0).fit(np.expand_dims(pix_abs, axis=1)),
        'gauss': GaussianMixture(n_components=2, random_state=0).fit(np.expand_dims(pix_abs, axis=1))
    },
    'hist-all': {
        'kmeans': KMeans(n_clusters=2, random_state=0).fit(np.expand_dims(hist_all, axis=1)),
        'gauss': GaussianMixture(n_components=2, random_state=0).fit(np.expand_dims(hist_all, axis=1))
    },
    'hist-reg': {
        'kmeans': KMeans(n_clusters=2, random_state=0).fit(np.expand_dims(hist_reg, axis=1)),
        'gauss': GaussianMixture(n_components=2, random_state=0).fit(np.expand_dims(hist_reg, axis=1))
    }
}

method_keys = ['abs', 'hist-all', 'hist-reg']
model_keys = ['kmeans', 'gauss']

In [4]:
method_key = 'hist-reg'
model_key = 'gauss'

model = model_dict[method_key][model_key]

if model_key == 'kmeans':
    no_change = min(model.cluster_centers_)[0]
else:
    no_change = min(model.means_)[0]

no_change_class_index = model.predict([[no_change]])[0]
print(no_change_class_index)

correct = 0
total = 0

for data in at_data:
    diff_score = atd.gen_pixel_diff(data, score_method=method_key)['pixel_diff_score']
    print(diff_score)
    preds = model.predict(np.expand_dims(diff_score, axis=1))
    print(preds)
    new_score = atd.non_max_suppression(diff_score, preds, no_change_class_index)
    print(new_score)
    new_pred = model.predict(np.expand_dims(new_score, axis=1))
    print(new_pred)
    pred_ab_change_list = [[j + 1, j + 2] for j, pred in enumerate(new_pred) if pred != no_change_class_index]
    print(pred_ab_change_list)
    gt_ab_change_list = data['scene_change_pairs']
    print(gt_ab_change_list)

    res = atd.ab_change_bin_acc(pred_ab_change_list, gt_ab_change_list)
    correct += res[0]
    total += res[1]
    
    break

0
[38526.0, 249238.0, 172990.0, 109608.0, 123498.0, 132102.0, 145818.0, 670712.0, 80416.0, 102220.0, 81488.0, 79064.0, 632366.0, 68782.0, 82584.0, 61380.0, 602840.0, 87782.0, 89490.0, 92746.0, 90538.0, 81114.0, 696390.0, 77038.0, 79440.0, 658352.0, 150800.0, 118946.0, 111366.0, 83110.0, 104054.0, 114834.0, 107826.0, 160216.0, 158684.0, 144574.0, 113628.0, 101312.0, 107324.0, 103218.0, 118464.0, 94114.0, 104342.0, 96970.0, 101482.0, 98328.0, 94990.0, 107126.0, 109962.0, 602222.0, 46166.0, 83900.0, 55492.0, 621598.0, 119168.0, 116384.0, 106200.0, 139690.0, 131226.0, 651144.0, 97156.0, 80202.0, 1029012.0, 604936.0, 314496.0, 282216.0, 1106862.0, 61720.0, 63076.0, 61008.0, 62354.0, 56928.0, 46666.0, 661356.0, 63756.0, 98908.0, 266544.0, 448274.0, 350470.0, 130122.0, 90470.0, 102844.0, 109404.0, 117398.0, 107664.0, 103844.0, 745768.0, 84564.0, 77952.0, 72288.0, 69706.0, 75964.0, 68638.0, 62268.0, 733938.0, 107240.0, 129598.0, 114980.0, 725510.0, 83736.0, 84106.0, 97082.0, 80264.0, 83778.0, 

In [5]:
def test_jacc(method_key: str, model_key: str):
    assert method_key in method_keys
    assert model_key in model_keys
    
    model = model_dict[method_key][model_key]

    if model_key == 'kmeans':
        no_change = min(model.cluster_centers_)[0]
    else:
        no_change = min(model.means_)[0]

    no_change_class_index = model.predict([[no_change]])[0]

    score = 0
    total = len(at_data)

    for data in at_data:
        diff_score = atd.gen_pixel_diff(data, score_method=method_key)['pixel_diff_score']
        preds = model.predict(np.expand_dims(diff_score, axis=1))
        new_score = atd.non_max_suppression(diff_score, preds, no_change_class_index)
        new_pred = model.predict(np.expand_dims(new_score, axis=1))
        pred_ab_change_list = [[j + 1, j + 2] for j, pred in enumerate(new_pred) if pred != no_change_class_index]
        gt_ab_change_list = data['scene_change_pairs']
        
        score += atd.ab_change_jacc_score(pred_ab_change_list, gt_ab_change_list, no_change_class_index)

    print(F'Jacc {method_key} - {model_key}')
    print(F'Jacc score:  {score}')
    print(F'Total score: {total}')
    print(F'Norm score:  {score / total}')
    print()


def test_bin(method_key: str, model_key: str):
    assert method_key in method_keys
    assert model_key in model_keys
    
    model = model_dict[method_key][model_key]

    if model_key == 'kmeans':
        no_change = min(model.cluster_centers_)[0]
    else:
        no_change = min(model.means_)[0]

    no_change_class_index = model.predict([[no_change]])[0]

    correct = 0
    total = 0

    for data in at_data:
        diff_score = atd.gen_pixel_diff(data, score_method=method_key)['pixel_diff_score']
        preds = model.predict(np.expand_dims(diff_score, axis=1))
        new_score = atd.non_max_suppression(diff_score, preds, no_change_class_index)
        new_pred = model.predict(np.expand_dims(new_score, axis=1))
        pred_ab_change_list = [[j + 1, j + 2] for j, pred in enumerate(new_pred) if pred != no_change_class_index]
        gt_ab_change_list = data['scene_change_pairs']
        
        res = atd.ab_change_bin_acc(pred_ab_change_list, gt_ab_change_list)
        correct += res[0]
        total += res[1]

    print(F'Bin {method_key} - {model_key}')
    print(F'Correct:  {correct}')
    print(F'Total:    {total}')
    print(F'Avg Acc:  {correct / total}')
    print()

In [6]:
for method_key in method_keys:
    for model_key in model_keys:
        test_jacc(method_key, model_key)

Jacc abs - kmeans
Jacc score:  3.3801597830665933
Total score: 10
Norm score:  0.3380159783066593

Jacc abs - gauss
Jacc score:  4.13150205651304
Total score: 10
Norm score:  0.413150205651304

Jacc hist-all - kmeans
Jacc score:  9.821613550615536
Total score: 10
Norm score:  0.9821613550615537

Jacc hist-all - gauss
Jacc score:  6.31907980177717
Total score: 10
Norm score:  0.631907980177717

Jacc hist-reg - kmeans
Jacc score:  9.36475848712691
Total score: 10
Norm score:  0.936475848712691

Jacc hist-reg - gauss
Jacc score:  0.8399102043697277
Total score: 10
Norm score:  0.08399102043697278



In [7]:
for method_key in method_keys:
    for model_key in model_keys:
        test_bin(method_key, model_key)

Bin abs - kmeans
Correct:  157
Total:    330
Avg Acc:  0.47575757575757577

Bin abs - gauss
Correct:  182
Total:    330
Avg Acc:  0.5515151515151515

Bin hist-all - kmeans
Correct:  299
Total:    330
Avg Acc:  0.906060606060606

Bin hist-all - gauss
Correct:  238
Total:    330
Avg Acc:  0.7212121212121212

Bin hist-reg - kmeans
Correct:  319
Total:    330
Avg Acc:  0.9666666666666667

Bin hist-reg - gauss
Correct:  165
Total:    330
Avg Acc:  0.5



In [8]:
# plt.figure(figsize=(18, 10))
# plt.hist(all_diff_score, bins=30)

# data = at_data[0]
# diff_list = diff_score_list[0]
# real_change = data['scene_change_pairs']

# all_pairs = [[i, i + 1] for i in range(1, 174)]
# for i, pair in enumerate(all_pairs):
#     if pair in real_change:
#         plt.axvline(diff_list[i], linewidth=1, color='k', linestyle='dashed')